In [ ]:
# Isolation Game with Alpha-Beta Pruning

import copy

BOARD_SIZE = 7
EMPTY = 0
BLOCKED = -1
PLAYER1 = 1
PLAYER2 = 2

DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1),
              (-1, -1), (-1, 1), (1, -1), (1, 1)]


def create_board():
    return [[EMPTY for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]


def print_board(board, p1_pos, p2_pos):
    display = [["." for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE):
            if board[r][c] == BLOCKED:
                display[r][c] = "X"
    if p1_pos:
        display[p1_pos[0]][p1_pos[1]] = "1"
    if p2_pos:
        display[p2_pos[0]][p2_pos[1]] = "2"
    for row in display:
        print(" ".join(row))
    print()


def get_valid_moves(board, position, opponent_pos=None):
    moves = []
    r, c = position
    for dr, dc in DIRECTIONS:
        nr, nc = r + dr, c + dc
        while 0 <= nr < BOARD_SIZE and 0 <= nc < BOARD_SIZE and board[nr][nc] == EMPTY:
            if (nr, nc) == opponent_pos:
                break  # Can't move into opponent's square
            moves.append((nr, nc))
            nr += dr
            nc += dc
    return moves

def make_move(board, from_pos, to_pos):
    new_board = copy.deepcopy(board)
    r,c=from_pos
    rt,ct=to_pos
    new_board[r][c]=BLOCKED
    return new_board

def evaluate(board, player_pos, opponent_pos):
    player_moves = len(get_valid_moves(board, player_pos))
    opponent_moves = len(get_valid_moves(board, opponent_pos))
    return player_moves - opponent_moves

def alphabeta(board, depth, alpha, beta, maximizing, player_pos, opponent_pos):
    if depth == 0 or is_game_over(board, player_pos, opponent_pos):
        return evaluate(board, player_pos, opponent_pos)

    if maximizing:
        max_eval = float("-inf")
        for move in get_valid_moves(board, player_pos, opponent_pos):
            new_board = make_move(board, player_pos, move)
            eval = alphabeta(new_board, depth - 1, alpha, beta, False, move, opponent_pos)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float("inf")
        for move in get_valid_moves(board, opponent_pos, player_pos):
            new_board = make_move(board, opponent_pos, move)
            eval = alphabeta(new_board, depth - 1, alpha, beta, True, player_pos, move)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def ai_move(board, player_pos, opponent_pos, depth=3):
    best_score = float("-inf")
    best_move = None
    for move in get_valid_moves(board, player_pos, opponent_pos):
        new_board = make_move(board, player_pos, move)
        score = alphabeta(new_board, depth - 1, float("-inf"), float("inf"), False, move, opponent_pos)
        if score > best_score:
            best_score = score
            best_move = move
    return best_move

def human_move(board, player_pos, p1_pos, p2_pos):
    valid = get_valid_moves(board, player_pos, p2_pos if player_pos == p1_pos else p1_pos)
    print("Valid moves:", valid)
    while True:
        r, c = map(int, input("Enter your move (row col): ").split())
        if (r, c) in valid:
            print(r,c)
            return (r, c)
        else:
            print("Invalid move.")
   
def is_game_over(board, player_pos, opponent_pos):
    return not get_valid_moves(board, player_pos, opponent_pos)

def play_game(ai_vs_ai=True):
    board = create_board()
    p1_pos = (0, 0)
    p2_pos = (6, 6)
    turn = PLAYER1

    while True:
        if turn == PLAYER1:
            print("============================")
            if is_game_over(board, p1_pos, p2_pos):
                print("Player 2 wins!")
                break
            if ai_vs_ai:
                move = ai_move(board, p1_pos, p2_pos)
            else:
                move = human_move(board, p1_pos, p1_pos, p2_pos)
            board = make_move(board, p1_pos, move)
            p1_pos = move
        else:
            if is_game_over(board, p2_pos, p1_pos):
                print("Player 1 wins!")
                break
            move = ai_move(board, p2_pos, p1_pos)
            board = make_move(board, p2_pos, move)
            p2_pos = move

        print_board(board, p1_pos, p2_pos)
        turn = PLAYER1 if turn == PLAYER2 else PLAYER2

    print("Game over!")

# Run the game (AI vs AI or set ai_vs_ai=False to play as Player 1)
play_game(ai_vs_ai=False)



Valid moves: [(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]
1 0
X . . . . . .
1 . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . 2

X . . . . . .
1 . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . 2
. . . . . . X

Valid moves: [(2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (0, 1), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5)]
6 5
X . . . . . .
X . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . 2
. . . . . 1 X

X . . . . . .
X . . . . . .
. . . . . . .
. . . . . . .
. . . . . . 2
. . . . . . X
. . . . . 1 X

Valid moves: [(5, 5), (4, 5), (3, 5), (2, 5), (1, 5), (0, 5), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0), (5, 4), (4, 3), (3, 2), (2, 1)]


In [2]:
print(1)

1
